# Notebook 1: Preparación y Análisis de Datos
**Clasificación Multi-Label - Pascal 2007**

**Clases objetivo:** person, chair, dog, sofa

Este notebook realiza:
- Instalación de dependencias
- Descarga del dataset Pascal 2007
- Análisis exploratorio de clases
- Filtrado de imágenes con clases objetivo
- Carga de imágenes y conversión a arrays numpy
- División en train/val/test sets

## 1. Instalación de Dependencias

In [ ]:
!pip install -q mlflow pillow ipywidgets scikit-learn pandas matplotlib tensorflow

## 2. Importar Librerías

In [ ]:
import os, gc, io, tarfile, urllib.request
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split

import tensorflow as tf
print(f"TensorFlow: {tf.__version__}")

# Verificar GPU
gpus = tf.config.list_physical_devices('GPU')
print(f"GPU: {gpus if gpus else 'No detectada'}")

## 3. Configuración Global

In [ ]:
# Configuración
IMG_SIZE = 224
BATCH_SIZE = 16
EPOCHS = 10
TARGET_CLASSES = ['person', 'chair', 'dog', 'sofa']
NUM_CLASSES = 4

# Directorio de datos
DATA_DIR = "pascal_2007"

# Guardar configuración en el directorio raíz del proyecto
# (para que utils.py y otros notebooks puedan acceder)
config = {
    'img_size': IMG_SIZE,
    'batch_size': BATCH_SIZE,
    'epochs': EPOCHS,
    'target_classes': TARGET_CLASSES,
    'num_classes': NUM_CLASSES
}

# Guardar en la raíz del proyecto Multi-Label_Classification
config_path = os.path.join('..', 'model_config.npy')
np.save(config_path, config)
print("✅ Configuración guardada en:", os.path.abspath(config_path))

## 4. Descargar Dataset Pascal 2007

In [ ]:
if not os.path.exists(DATA_DIR):
    print("Descargando dataset Pascal 2007...")
    urllib.request.urlretrieve("https://s3.amazonaws.com/fast-ai-imagelocal/pascal_2007.tgz", "pascal.tgz")
    print("Extrayendo archivos...")
    with tarfile.open("pascal.tgz", "r:gz") as tar:
        tar.extractall()
    os.remove("pascal.tgz")
    print("✅ Dataset descargado y extraído")
else:
    print("✅ Dataset ya existe")

## 5. Análisis Exploratorio del Dataset

In [ ]:
# Cargar CSV con etiquetas
df = pd.read_csv(os.path.join(DATA_DIR, "train.csv"))
print(f"Total de imágenes en el dataset: {len(df)}")
print(f"\nPrimeras filas:")
print(df.head())

# Contar todas las clases disponibles
all_labels = set()
for lbls in df['labels'].str.split():
    all_labels.update(lbls)
    
print(f"\nTotal de clases disponibles: {len(all_labels)}")
print(f"Clases: {sorted(all_labels)}")

## 6. Visualizar Distribución de Clases

In [ ]:
# Contar frecuencia de cada clase
counts = {l: sum(1 for x in df['labels'] if l in x.split()) for l in all_labels}

# Visualizar
plt.figure(figsize=(14, 5))
plt.bar(counts.keys(), counts.values())
plt.xticks(rotation=45, ha='right')
plt.title('Distribución de Todas las Clases en Pascal 2007', fontsize=14)
plt.xlabel('Clase')
plt.ylabel('Frecuencia')
plt.tight_layout()
plt.show()

print(f"\nClases más frecuentes:")
for label, count in sorted(counts.items(), key=lambda x: x[1], reverse=True)[:10]:
    print(f"  {label}: {count}")

## 7. Filtrar Imágenes con Clases Objetivo

In [ ]:
def has_target_class(labels_str):
    """Verifica si la imagen tiene al menos una clase objetivo."""
    return any(l in TARGET_CLASSES for l in labels_str.split())

def get_label_vector(labels_str):
    """Convierte string de etiquetas a vector binario."""
    lbls = labels_str.split()
    return [1.0 if c in lbls else 0.0 for c in TARGET_CLASSES]

# Filtrar dataset
filtered_df = df[df['labels'].apply(has_target_class)].reset_index(drop=True)
print(f"Imágenes filtradas con clases objetivo: {len(filtered_df)} de {len(df)} ({len(filtered_df)/len(df)*100:.1f}%)")

# Contar clases objetivo
target_counts = {l: sum(1 for x in filtered_df['labels'] if l in x.split()) for l in TARGET_CLASSES}
print(f"\nDistribución de clases objetivo:")
for label, count in target_counts.items():
    print(f"  {label}: {count}")

## 8. Visualizar Distribución de Clases Objetivo

In [ ]:
# Gráfico de barras
plt.figure(figsize=(10, 5))
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
plt.bar(target_counts.keys(), target_counts.values(), color=colors)
plt.title('Distribución de Clases Objetivo', fontsize=14)
plt.xlabel('Clase')
plt.ylabel('Frecuencia')
plt.tight_layout()
plt.show()

## 9. Cargar Imágenes como Arrays Numpy

In [ ]:
img_dir = os.path.join(DATA_DIR, "train")

# Cargar TODAS las imágenes y etiquetas como numpy arrays
all_images = []
all_labels = []

print("Cargando imágenes...")
for i, row in filtered_df.iterrows():
    img_path = os.path.join(img_dir, row['fname'])
    if os.path.exists(img_path):
        # Cargar, redimensionar y normalizar imagen
        img = Image.open(img_path).convert('RGB').resize((IMG_SIZE, IMG_SIZE))
        all_images.append(np.array(img, dtype=np.float32) / 255.0)
        all_labels.append(get_label_vector(row['labels']))
        
    if (i + 1) % 500 == 0:
        print(f"  Procesadas: {i + 1}/{len(filtered_df)}")

# Convertir a numpy arrays
X_all = np.array(all_images, dtype=np.float32)
y_all = np.array(all_labels, dtype=np.float32)

print(f"\n✅ Datos cargados:")
print(f"   X shape: {X_all.shape} (imágenes x altura x ancho x canales)")
print(f"   y shape: {y_all.shape} (imágenes x clases)")
print(f"   X dtype: {X_all.dtype}")
print(f"   y dtype: {y_all.dtype}")

## 10. Visualizar Ejemplos de Imágenes

In [ ]:
# Mostrar algunas imágenes de ejemplo
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()

for i in range(10):
    idx = np.random.randint(0, len(X_all))
    axes[i].imshow(X_all[idx])
    labels = [TARGET_CLASSES[j] for j, v in enumerate(y_all[idx]) if v == 1.0]
    axes[i].set_title(f"{', '.join(labels)}")
    axes[i].axis('off')

plt.suptitle('Ejemplos de Imágenes con sus Etiquetas', fontsize=16)
plt.tight_layout()
plt.show()

## 11. División de Datos (Train/Val/Test)

In [ ]:
# Dividir datos: 70% train, 15% val, 15% test
X_train, X_temp, y_train, y_temp = train_test_split(
    X_all, y_all, test_size=0.3, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

print("✅ División de datos completada:")
print(f"   Train: {X_train.shape[0]} imágenes ({X_train.shape[0]/len(X_all)*100:.1f}%)")
print(f"   Val:   {X_val.shape[0]} imágenes ({X_val.shape[0]/len(X_all)*100:.1f}%)")
print(f"   Test:  {X_test.shape[0]} imágenes ({X_test.shape[0]/len(X_all)*100:.1f}%)")

## 12. Verificar Tipos de Datos

In [ ]:
# Verificación crítica - asegurar que no hay strings
print("Verificando tipos de datos:")
print(f"  X_train: {X_train.dtype} (debe ser float32)")
print(f"  y_train: {y_train.dtype} (debe ser float32)")
print(f"  X_val:   {X_val.dtype}")
print(f"  y_val:   {y_val.dtype}")
print(f"  X_test:  {X_test.dtype}")
print(f"  y_test:  {y_test.dtype}")

assert X_train.dtype == np.float32, "ERROR: X_train no es float32"
assert y_train.dtype == np.float32, "ERROR: y_train no es float32"
print("\n✅ Todos los datos son float32 - listos para entrenamiento")

## 13. Guardar Datos Procesados

In [ ]:
# Guardar datos procesados para uso en otros notebooks
data_output_dir = "processed_data"
os.makedirs(data_output_dir, exist_ok=True)

print("Guardando datos procesados...")
np.save(os.path.join(data_output_dir, 'X_train.npy'), X_train)
np.save(os.path.join(data_output_dir, 'y_train.npy'), y_train)
np.save(os.path.join(data_output_dir, 'X_val.npy'), X_val)
np.save(os.path.join(data_output_dir, 'y_val.npy'), y_val)
np.save(os.path.join(data_output_dir, 'X_test.npy'), X_test)
np.save(os.path.join(data_output_dir, 'y_test.npy'), y_test)

print(f"\n✅ Datos guardados en: {data_output_dir}/")
print("\n📝 Resumen:")
print(f"   - Dataset: Pascal 2007")
print(f"   - Clases objetivo: {TARGET_CLASSES}")
print(f"   - Tamaño de imagen: {IMG_SIZE}x{IMG_SIZE}")
print(f"   - Total imágenes: {len(X_all)}")
print(f"   - Train/Val/Test: {len(X_train)}/{len(X_val)}/{len(X_test)}")

## 14. Liberar Memoria

In [ ]:
# Liberar memoria de variables temporales
del all_images, all_labels, X_all, y_all, X_temp, y_temp
gc.collect()
print("✅ Memoria liberada")

---
##  Notebook 1 Completado
